In [33]:
def minimize_dfa(dfa):
    # Step 1: Initial partition (final vs non-final states)
    P = [set(dfa["final_states"]), set(dfa["states"]) - set(dfa["final_states"])]

    # Step 2: Iteration counter
    k = 1

    # Step 3: Refine partition until no change
    while True:
        Pk = []
        for partition in P:
            # No need to split sets with single state
            if len(partition) == 1:
                Pk.append(partition)
                continue

            # Check distinguishability for all state pairs
            split = False
            for q1 in partition:
                for q2 in partition:
                    if q1 != q2 and is_distinguishable(dfa, q1, q2):
                        split = True
                        break
                if split:
                    break

            if split:
                Pk.extend([{q} for q in partition])  # Split into singleton sets
            else:
                Pk.append(partition)  # Keep set unchanged

        # Check for convergence
        if Pk == P:
            break

        P = Pk
        k += 1

    # Step 4: Construct minimized DFA
    minimized_dfa = {
        "states": set.union(*P),
        "alphabet": dfa["alphabet"],
        "delta": {},
        "start_state": None,
        "final_states": None
    }

    # Map old states to new state labels (based on order in P)
    state_map = {q: i for i, partition in enumerate(P) for q in partition}

    # Build transition function and identify start/final states
    for q, transitions in dfa["delta"].items():
        for symbol, next_state in transitions.items():
            minimized_dfa["delta"][state_map[q], symbol] = state_map[next_state]
        minimized_dfa["start_state"] = state_map[dfa["start_state"]]
        minimized_dfa["final_states"] = {state_map[q] for q in dfa["final_states"]}

    return minimized_dfa

In [35]:
def is_distinguishable(dfa, q1, q2):
    for symbol in dfa["alphabet"]:
        if dfa["delta"][q1][symbol] != dfa["delta"][q2][symbol]:
            return True
    return False

In [36]:
# Example usage (replace with your DFA definition)
dfa = {
    "states": {"q0", "q1", "q2", "q3"},
    "alphabet": {"0", "1"},
    "delta": {
        "q0": {"0": "q1", "1": "q2"},
        "q1": {"0": "q3", "1": "q2"},
        "q2": {"0": "q3", "1": "q1"},
        "q3": {"0": "q0", "1": "q0"}
    },
    "start_state": "q0",
    "final_states": {"q2"}
}

minimized_dfa = minimize_dfa(dfa)
print(minimized_dfa)

{'states': {'q2', 'q1', 'q0', 'q3'}, 'alphabet': {'1', '0'}, 'delta': {(2, '0'): 1, (2, '1'): 0, (1, '0'): 3, (1, '1'): 0, (0, '0'): 3, (0, '1'): 1, (3, '0'): 2, (3, '1'): 2}, 'start_state': 2, 'final_states': {0}}
